# Exploring Listwise ranking models

Steps:
1. Generate random query data
   1. Generate true relevance scores from a classification problem, proba as the scores.
   2. Convert set of documents and scores into labelled query dataset, where only one document has a binary positive label
2. Convert to listwise data representation
3. Train ranker models to estimate the true relevance scores
   1. Pointwise
   2. Pairwise
   3. Listwise

References:
1. LambdaLoss - https://dl.acm.org/doi/10.1145/3269206.3271784
2. https://towardsdatascience.com/learning-to-rank-a-complete-guide-to-ranking-using-machine-learning-4c9688d370d4

Todo:
1. Try different query sizes
2. Introduce ranking/position bias
3. Apply methods to real datasets, like Yahoo LTRC
   1. https://www.csie.ntu.edu.tw/~cjlin/libsvmtools/datasets/

## Learn to rank formulation
We follow the approach outlined by the paper ["The LambdaLoss Framework for Ranking Metric Optimization" 2018](https://dl.acm.org/doi/10.1145/3269206.3271784).


We have a training data set, $T$, comprising of $Q$ queries and for each query we have a list of documents $d$.
Each document in the query has a feature vector and a relevance label.
Each query is represented with a list of document features, $\mathbf{x}_q$, and a list of relevance labels, $\mathbf{y}_q$.
The training dataset is notated as:
$$T = \{(\mathbf{x}_q, \mathbf{y}_q) | q \in Q\}$$
We drop $q$ for brevity going forwards.

A learning to rank model aims to find the underlying relevance scores, $\mathbf{s}$, in each query:
$$\mathbf{s} = f(\mathbf{x})$$
These scores are typically not observed. For a given query the predicted document ranking is given by ordering with the document scores.

We define a loss function, $l: (\mathbf{y},\mathbf{s}) \rightarrow \mathbb{R}$, to capture how well our model performs given the relevance labels, $\mathbf{y}_q$.
This is summed over all queries:
$$\mathcal{L}(f) = \frac{1}{|T|} \sum_{(\mathbf{x},\mathbf{y}) \in T} l(\mathbf{y},f(\mathbf{x}))$$


### Applications to click data
The relevance label we will treat as a click. Given a query only a single document can have a click, $\sum_i y_{q,i} \leq 1$.

In [1]:
import numpy as np
import pandas as pd
import scipy.stats
import matplotlib.pyplot as plt
import seaborn as sns

plt.style.use("seaborn-whitegrid")

np.random.seed(0)

/var/folders/_v/nlh4h1yx2n1gd6f3szjlgxt40000gr/T/ipykernel_25240/3873074974.py:7: MatplotlibDeprecationWarning: The seaborn styles shipped by Matplotlib are deprecated since 3.6, as they no longer correspond to the styles shipped by seaborn. However, they will remain available as 'seaborn-v0_8-<style>'. Alternatively, directly use the seaborn API instead.
  plt.style.use("seaborn-whitegrid")


## Generate data

We generate relevance scores and use them to rank documents within queries.

First we generate true relevance scores using a regression based data generating function:

In [2]:
import sklearn.datasets

n_queries = 10_000
n_documents_per_query = 5
n_features = 5

X, s_true = sklearn.datasets.make_regression(
    n_samples=n_queries * n_documents_per_query, n_features=n_features, noise=10
)

Now we create a list of queries and create relevance labels from the relevance scores.

In [3]:
import polars as pl

p_click = 0.9

q_id = np.repeat(np.arange(n_queries), repeats=n_documents_per_query)

df = pl.DataFrame(data=X, schema=[f"f_{idx}" for idx in range(n_features)])
df = (
    df.with_columns([pl.lit(q_id).alias("q_id"), pl.lit(s_true).alias("s_true")])
    .with_columns(pl.lit(0).alias("doc_id"))
    .with_columns(pl.col("doc_id").cumcount().over("q_id"))
    .with_columns(
        (
            (pl.col("s_true").arg_max().over(pl.col("q_id")) == pl.col("doc_id")) * pl.lit(1.0* (np.random.uniform(size=(n_queries*n_documents_per_query))<p_click))
        ).alias("relevance_label")
    )
)
df


f_0,f_1,f_2,f_3,f_4,q_id,s_true,doc_id,relevance_label
f64,f64,f64,f64,f64,i64,f64,u32,f64
0.942773,0.126148,0.879036,0.806536,0.986878,0,95.468621,0,0.0
-0.049142,-1.833773,2.015876,0.317939,0.061935,0,-19.85206,1,0.0
0.622839,-1.01906,1.696835,-0.895207,-0.701512,0,-20.654428,2,0.0
-2.181628,-1.022559,-0.324561,-1.621306,-0.062113,0,-166.629593,3,0.0
0.385964,0.345259,-2.229314,2.126856,0.797797,0,17.019736,4,0.0
-0.572585,-2.455971,-0.968181,0.056421,1.252214,1,-95.528365,0,0.0
0.646054,1.265322,0.0897,-0.284192,0.856051,1,97.36738,1,1.0
-0.393146,0.235448,-0.902952,-0.368897,-0.665762,1,-43.269676,2,0.0
-0.841348,-0.073625,1.475709,-1.172488,-0.702269,1,-40.718733,3,0.0


## Fit models

In [4]:
df_train = df.head(int(len(df) / 2))
df_test = df.tail(int(len(df) / 2))

x_cols = [f"f_{idx}" for idx in range(n_features)]
y_col = "relevance_label"

models = {}

### Pointwise

LightGBM - GBM model predicting just the relevance labels from features.
This ignores any relationship between documents.

LightGBM install on M1 mac:
https://github.com/microsoft/LightGBM/issues/6035#issuecomment-1676415781



In [5]:
import sklearn.linear_model

models["pointwise_log"] = sklearn.linear_model.LogisticRegression()
models["pointwise_log"].fit(X=df_train[x_cols].to_pandas(), y=df_train[y_col].to_pandas())

LogisticRegression()

In [6]:
import sklearn.ensemble

models["pointwise_gbm"] = sklearn.ensemble.HistGradientBoostingRegressor()
models["pointwise_gbm"].fit(X=df_train[x_cols].to_pandas(), y=df_train[y_col].to_pandas())

HistGradientBoostingRegressor()

In [7]:
import lightgbm as lgb

models["pointwise_lgb"] = lgb.LGBMRegressor()
models["pointwise_lgb"].fit(
    X=df_train[x_cols].to_pandas(), y=df_train[y_col].to_pandas()
)

LGBMRegressor()

Pointwise model using the features of all documents in the query

This will not scale well to large document numbers.

We pivot the features and use doc_id as the distinguishing feature.

In [8]:
def pivot_features(df, x_cols, return_piv_x_cols=False):
    df_piv = df.pivot(values=x_cols, index="q_id", columns="doc_id", aggregate_function=None)
    piv_x_cols = ['doc_id']+ df_piv.columns
    piv_x_cols.remove("q_id")
    if return_piv_x_cols:
        return df.join(df_piv, on="q_id"), piv_x_cols
    return df.join(df_piv, on="q_id")


In [9]:
import lightgbm as lgb

df_piv, piv_x_cols = pivot_features(df_train, x_cols, return_piv_x_cols=True)
models["pointwise_lgb_piv"] = lgb.LGBMRegressor()
models["pointwise_lgb_piv"].fit(
    X=df_piv[piv_x_cols].to_pandas(), y=df_train[y_col].to_pandas()
)

LGBMRegressor()

### List/Pairwise

In [10]:
import lightgbm as lgb

models["pointwise_ranker"] = lgb.LGBMRanker(objective="lambdarank")
group_counts = df_train.groupby(["q_id"]).count().sort("q_id")["count"].to_pandas()
models["pointwise_ranker"].fit(
    X=df_train[x_cols].to_pandas(), y=df_train[y_col].to_pandas(), group=group_counts
)

LGBMRanker(objective='lambdarank')

XGBoost interface

In [13]:
import xgboost as xgb

models["pointwise_ranker_xgb"] = xgb.XGBRanker(
    tree_method="hist",
    lambdarank_num_pair_per_sample=8,
    objective="rank:ndcg",
    lambdarank_pair_method="mean",
)
models["pointwise_ranker_xgb"].fit(X=df_train[x_cols].to_pandas(), y=df_train[y_col].to_pandas(), qid=df_train["q_id"].to_pandas())

XGBRanker(base_score=None, booster=None, callbacks=None, colsample_bylevel=None,
          colsample_bynode=None, colsample_bytree=None, device=None,
          early_stopping_rounds=None, enable_categorical=False,
          eval_metric=None, feature_types=None, gamma=None, grow_policy=None,
          importance_type=None, interaction_constraints=None,
          lambdarank_num_pair_per_sample=8, lambdarank_pair_method='mean',
          learning_rate=None, max_bin=None, max_cat_threshold=None,
          max_cat_to_onehot=None, max_delta_step=None, max_depth=None,
          max_leaves=None, min_child_weight=None, missing=nan,
          monotone_constraints=None, multi_strategy=None, n_estimators=None,
          n_jobs=None, ...)

In [16]:
models["pointwise_ranker_xgb"].predict(X=df_train[x_cols].to_pandas())

array([ 3.852983, -4.580815, -5.106556, ..., -4.800625, -9.056662,
        3.373502], dtype=float32)

In [23]:
models["pointwise_ranker_xgb"].predict(X=df_test[x_cols].to_pandas())

array([ -6.8877397,  -3.8244393,   0.2314953, ..., -11.10918  ,
        -2.209039 ,  -7.083221 ], dtype=float32)

## Evaluation

Predict scores for each document

Group up and find predicted ranking.
Compare ranking to relevance labels to get NDCG for each case.

In [24]:
import sklearn.metrics


def _ndcg_calc(df):
    ndcg = sklearn.metrics.ndcg_score(
        y_true=df["relevance_label"].to_numpy()[np.newaxis, :],
        y_score=df["s_pred"].to_numpy()[np.newaxis, :],
    )
    return pl.DataFrame(data={"q_id": df["q_id"].head(1), "ndcg": ndcg})


def calculate_ndcg(df: pl.DataFrame, s_pred):
    df = df.with_columns(pl.lit(s_pred).alias("s_pred"))
    df = df.groupby("q_id").apply(_ndcg_calc).sort("q_id")
    return df["ndcg"].mean()


print(calculate_ndcg(df_test, models["pointwise_log"].predict(df_test[x_cols])))
print(calculate_ndcg(df_test, models["pointwise_gbm"].predict(df_test[x_cols])))
print(calculate_ndcg(df_test, models["pointwise_lgb"].predict(df_test[x_cols])))
print(calculate_ndcg(df_test, models["pointwise_lgb_piv"].predict(pivot_features(df_test, x_cols)[piv_x_cols])))  # fmt: skip
print(calculate_ndcg(df_test, models["pointwise_ranker"].predict(df_test[x_cols])))
print(calculate_ndcg(df_test, models["pointwise_ranker_xgb"].predict(X=df_test[x_cols].to_pandas())))  # fmt: skip

0.6869805354515948
0.8426202511320225
0.8391898435791719
0.7791243586995159
0.8521843347164926
0.8503613491425793


# Appendix

In [12]:
raise NotImplementedError

NotImplementedError: 

Testing predict on lgb, do we need to do single group at a time - no.
Model predictions are independent to get a predicted score. The ranking is simply on the predicted score values.

In [ ]:
s_pred = []
for _qid in df_test['q_id'].unique():
    s_pred.extend(models["pointwise_ranker"].predict(df_test.filter(pl.col('q_id')==_qid)[x_cols]))

In [ ]:
np.sum(np.abs(np.array(s_pred) - models["pointwise_ranker"].predict(df_test[x_cols])))

0.0

Generate data from queries

using a set number of documents for each query.

In [ ]:
n_queries = 10
n_documents = 5
n_documents_per_query = 3

rnd = np.random.default_rng()

# document IDs
query_data = np.array(
    [
        rnd.choice(np.arange(n_documents), size=(n_documents_per_query), replace=False)
        for _q in range(n_queries)
    ]
)
# labels are taken as the action on the document ID
labels_data = query_data[
    np.arange(n_queries), rnd.choice(np.arange(n_documents_per_query), size=(n_queries))
]

df = pd.DataFrame(
    data=np.concatenate(
        [np.arange(n_queries)[:, np.newaxis], query_data, labels_data[:, np.newaxis]],
        axis=1,
    ),
    columns=["query_id"]
    + [f"doc_{_idx}" for _idx in range(n_documents_per_query)]
    + ["label"],
)
df

,query_id,doc_0,doc_1,doc_2,label
0,0,2,3,4,3
1,1,2,3,0,3
2,2,2,0,1,1
3,3,4,0,1,4
4,4,3,2,0,0
5,5,0,2,3,0
6,6,2,0,4,4
7,7,1,4,0,1
8,8,4,1,2,4
9,9,0,2,3,0


Generate listwise data

In [ ]:
for _idx, row in df.iterrows():
    row


# df_melt = df.melt(
#     id_vars="query_id",
#     value_vars=[f"doc_{_idx}" for _idx in range(n_documents_per_query)],
#     var_name="doc_id",
# ).sort_values(["query_id", "doc_id"])
row

query_id    9
doc_0       0
doc_1       2
doc_2       3
label       0
Name: 9, dtype: int64

Generate data from queries in listwise form

using a set number of documents for each query.

In [ ]:
n_queries = 10
n_documents = 5
n_documents_per_query = 3

rnd = np.random.default_rng()

document_rows = []
for _q_id in range(n_queries):
    doc_ids = rnd.choice(
        np.arange(n_documents), size=(n_documents_per_query), replace=False
    )
    labels = rnd.uniform(size=(n_documents_per_query))
    labels = (labels == np.max(labels)) * 1
    for doc_id, label in zip(doc_ids, labels):
        document_rows.append({"query_id": _q_id, "doc_id": doc_id, "label": label})

df = pd.DataFrame(document_rows)
df

,query_id,doc_id,label
0,0,1,1
1,0,0,0
2,0,3,0
3,1,4,0
4,1,0,0
5,1,1,1
6,2,1,1
7,2,0,0
8,2,4,0
9,3,3,1


Build rankers

Assume that doc_id is the only feature (ordinal encoding)

In [ ]:
import xgboost as xgb

In [ ]:
import xgboost as xgb

ranker = xgb.XGBRanker(
    tree_method="hist",
    lambdarank_num_pair_per_sample=8,
    objective="rank:ndcg",
    lambdarank_pair_method="mean",
)
ranker.fit(X=df[["doc_id"]], y=df["label"], qid=df["query_id"])

XGBRanker(base_score=None, booster=None, callbacks=None, colsample_bylevel=None,
          colsample_bynode=None, colsample_bytree=None, device=None,
          early_stopping_rounds=None, enable_categorical=False,
          eval_metric=None, feature_types=None, gamma=None, grow_policy=None,
          importance_type=None, interaction_constraints=None,
          lambdarank_num_pair_per_sample=8, lambdarank_pair_method='mean',
          learning_rate=None, max_bin=None, max_cat_threshold=None,
          max_cat_to_onehot=None, max_delta_step=None, max_depth=None,
          max_leaves=None, min_child_weight=None, missing=nan,
          monotone_constraints=None, multi_strategy=None, n_estimators=None,
          n_jobs=None, ...)

Predictions

In [ ]:
scores = ranker.predict(X=df[["doc_id"]])
df = df.assign(scores=scores)
df

,query_id,doc_id,label,scores
0,0,1,1,0.619957
1,0,0,0,-0.193321
2,0,3,0,0.619366
3,1,4,0,-0.640745
4,1,0,0,-0.193321
5,1,1,1,0.619957
6,2,1,1,0.619957
7,2,0,0,-0.193321
8,2,4,0,-0.640745
9,3,3,1,0.619366


Predictions by doc_id are always the same...?

In [ ]:
df.groupby(["doc_id"])["scores"].agg(["max", "min", "count"])

,max,min,count
doc_id,,,
0,-0.193321,-0.193321,7
1,0.619957,0.619957,6
2,-2.902935,-2.902935,5
3,0.619366,0.619366,6
4,-0.640745,-0.640745,6


Counterfactual predictions

Estimate doc_id 1 relevance in queries of different lengths
Relevance score for query_id/doc_id pairs are the same...?

In [ ]:
df_rand = df.sample(frac=0.5, replace=False).sort_values(["query_id", "doc_id"])
scores = ranker.predict(X=df_rand[["doc_id"]])
df_rand = df_rand.assign(scores=scores)

df_rand.merge(df, on=["query_id", "doc_id", "label"], how="outer").sort_values(
    ["query_id", "doc_id"]
).assign(diff=lambda x: x["scores_y"] - x["scores_x"])

,query_id,doc_id,label,scores_x,scores_y,diff
15,0,0,0,NaN,-0.193321,NaN
0,0,1,1,0.619957,0.619957,0.0
16,0,3,0,NaN,0.619366,NaN
18,1,0,0,NaN,-0.193321,NaN
1,1,1,1,0.619957,0.619957,0.0
17,1,4,0,NaN,-0.640745,NaN
20,2,0,0,NaN,-0.193321,NaN
19,2,1,1,NaN,0.619957,NaN
2,2,4,0,-0.640745,-0.640745,0.0
3,3,2,0,-2.902935,-2.902935,0.0


Counterfactual predictions

Change the doc_ids for half of the queries

In [ ]:
df_rand = df.sample(frac=0.5, replace=False).sort_values(["query_id", "doc_id"])
scores = ranker.predict(X=df_rand[["doc_id"]])
df_rand = df_rand.assign(scores=scores)

df_rand.merge(df, on=["query_id", "doc_id", "label"], how="outer").sort_values(
    ["query_id", "doc_id"]
).assign(diff=lambda x: x["scores_y"] - x["scores_x"])

,query_id,doc_id,label,scores_x,scores_y,diff
15,0,0,0,NaN,-0.193321,NaN
0,0,1,1,0.619957,0.619957,0.0
16,0,3,0,NaN,0.619366,NaN
1,1,0,0,-0.193321,-0.193321,0.0
17,1,1,1,NaN,0.619957,NaN
2,1,4,0,-0.640745,-0.640745,0.0
3,2,0,0,-0.193321,-0.193321,0.0
18,2,1,1,NaN,0.619957,NaN
19,2,4,0,NaN,-0.640745,NaN
4,3,2,0,-2.902935,-2.902935,0.0


In [ ]:
from sklearn.datasets import make_classification
import numpy as np

import xgboost as xgb

# Make a synthetic ranking dataset for demonstration
seed = 1994
X, y = make_classification(random_state=seed)
rng = np.random.default_rng(seed)
n_query_groups = 3
qid = rng.integers(0, 3, size=X.shape[0])

# Sort the inputs based on query index
sorted_idx = np.argsort(qid)
X = X[sorted_idx, :]
y = y[sorted_idx]
qid = qid[sorted_idx]